# A4. クエリとACLデモ (Graph API パターン)

## 概要

このノートブックでは、Azure AI Search に取り込んだデータに対して様々な検索クエリを実行します:

1. **キーワード検索**: 基本的なテキスト検索
2. **セマンティック検索**: 意味ベースの検索
3. **ベクター検索**: 埋め込みベクトルによる類似検索
4. **ハイブリッド検索**: キーワード + ベクター検索の組み合わせ
5. **ACL フィルタ**: ユーザー/グループ権限による検索結果フィルタリング

## 前提条件

- `A2_graph_ingest_run.ipynb` でデータが取り込まれていること
- Azure AI Search インデックスが存在すること

## 1. 環境の初期化

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Dict, Optional

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential

# OpenAI (埋め込み生成用、オプション)
try:
    from openai import AzureOpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAI SDK が利用できません。ベクター検索はスキップされます。")

# 環境変数の読み込み
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env ファイルを読み込みました")
else:
    print("⚠️ .env ファイルが見つかりません")

# 必須環境変数の確認
required_vars = ["AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_INDEX_NAME"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"環境変数が設定されていません: {', '.join(missing_vars)}")

print("\n✅ 環境変数の確認完了")

## 2. Search クライアントの初期化

In [ ]:
# Search クライアントの作成
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

print(f"✅ Search クライアント作成完了")
print(f"   エンドポイント: {search_endpoint}")
print(f"   インデックス名: {index_name}")

# ドキュメント数の確認
try:
    results = search_client.search(search_text="*", top=1, include_total_count=True)
    total_docs = results.get_count()
    print(f"\n📊 インデックス内のドキュメント数: {total_docs}")
except Exception as e:
    print(f"⚠️ ドキュメント数の取得に失敗: {e}")

## 3. OpenAI クライアントの初期化 (ベクター検索用、オプション)

In [ ]:
openai_client = None
embedding_deployment = None

if OPENAI_AVAILABLE and os.getenv("AZURE_OPENAI_ENDPOINT"):
    try:
        openai_client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
        )
        embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
        print("✅ OpenAI クライアント作成完了")
        print(f"   デプロイメント: {embedding_deployment}")
    except Exception as e:
        print(f"⚠️ OpenAI クライアントの作成に失敗: {e}")
        openai_client = None
else:
    print("ℹ️ OpenAI 設定がありません。キーワード検索とセマンティック検索のみ利用可能です。")

## 4. ヘルパー関数

In [ ]:
def generate_embedding(text: str) -> Optional[List[float]]:
    """
    テキストから埋め込みベクトルを生成
    
    Args:
        text: 埋め込みを生成するテキスト
    
    Returns:
        埋め込みベクトル (利用できない場合は None)
    """
    if not openai_client or not embedding_deployment:
        return None
    
    try:
        response = openai_client.embeddings.create(
            input=text,
            model=embedding_deployment
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"⚠️ 埋め込み生成エラー: {e}")
        return None


def print_search_results(results, max_results: int = 5, show_content: bool = True):
    """
    検索結果を見やすく表示
    
    Args:
        results: 検索結果
        max_results: 表示する最大結果数
        show_content: 本文を表示するか
    """
    count = 0
    for result in results:
        if count >= max_results:
            break
        
        print(f"\n{'='*80}")
        print(f"📄 結果 #{count + 1}")
        print(f"{'='*80}")
        
        # スコア
        score = result.get('@search.score')
        if score:
            print(f"スコア: {score:.4f}")
        
        # タイトル
        title = result.get('title', 'タイトルなし')
        print(f"タイトル: {title}")
        
        # URL
        url = result.get('url')
        if url:
            print(f"URL: {url}")
        
        # メタデータ
        content_type = result.get('contentType')
        if content_type:
            print(f"種類: {content_type}")
        
        last_modified = result.get('lastModified')
        if last_modified:
            print(f"更新日時: {last_modified}")
        
        # セマンティックキャプション
        captions = result.get('@search.captions')
        if captions:
            print(f"\n📝 キャプション:")
            for caption in captions[:2]:  # 最大2つまで表示
                caption_text = caption.get('text', '')
                if caption_text:
                    print(f"  - {caption_text[:200]}...")
        
        # 本文
        if show_content:
            content = result.get('content', '')
            if content:
                # 最初の200文字を表示
                preview = content[:200] + "..." if len(content) > 200 else content
                print(f"\n📖 本文プレビュー:\n{preview}")
        
        count += 1
    
    if count == 0:
        print("\n検索結果が見つかりませんでした。")
    else:
        print(f"\n\n表示: {count} 件")

## 5. キーワード検索

基本的なテキスト検索を実行します。

In [ ]:
# 検索クエリ
search_query = "契約書"

print(f"🔍 キーワード検索: '{search_query}'\n")

try:
    results = search_client.search(
        search_text=search_query,
        top=5,
        select=["id", "title", "content", "url", "contentType", "lastModified"]
    )
    
    print_search_results(results, max_results=5, show_content=True)
    
except Exception as e:
    print(f"❌ 検索エラー: {e}")

## 6. セマンティック検索

意味ベースの検索を実行します。Azure AI Search のセマンティック検索機能を使用します。

**注意**: セマンティック検索を使用するには、インデックスにセマンティック設定が必要です。

In [ ]:
# セマンティック検索クエリ
semantic_query = "契約の更新手順について教えて"

print(f"🧠 セマンティック検索: '{semantic_query}'\n")

try:
    results = search_client.search(
        search_text=semantic_query,
        query_type="semantic",
        semantic_configuration_name="default",  # インデックスのセマンティック設定名に応じて変更
        top=5,
        select=["id", "title", "content", "url", "contentType", "lastModified"]
    )
    
    print_search_results(results, max_results=5, show_content=True)
    
except Exception as e:
    print(f"⚠️ セマンティック検索エラー: {e}")
    print("ℹ️ セマンティック検索を使用するには、インデックスにセマンティック設定が必要です。")

## 7. ベクター検索

埋め込みベクトルを使った類似検索を実行します。

In [ ]:
# ベクター検索クエリ
vector_query_text = "プロジェクトの進捗報告"

print(f"🎯 ベクター検索: '{vector_query_text}'\n")

if openai_client:
    try:
        # クエリの埋め込みを生成
        query_vector = generate_embedding(vector_query_text)
        
        if query_vector:
            # ベクター検索の実行
            vector_query = VectorizedQuery(
                vector=query_vector,
                k_nearest_neighbors=5,
                fields="contentVector"  # インデックスのベクターフィールド名に応じて変更
            )
            
            results = search_client.search(
                search_text=None,
                vector_queries=[vector_query],
                top=5,
                select=["id", "title", "content", "url", "contentType", "lastModified"]
            )
            
            print_search_results(results, max_results=5, show_content=True)
        else:
            print("❌ 埋め込みベクトルの生成に失敗しました")
    
    except Exception as e:
        print(f"⚠️ ベクター検索エラー: {e}")
        print("ℹ️ ベクター検索を使用するには、インデックスにベクターフィールドが必要です。")
else:
    print("ℹ️ OpenAI クライアントが利用できません。ベクター検索をスキップします。")

## 8. ハイブリッド検索

キーワード検索とベクター検索を組み合わせたハイブリッド検索を実行します。

In [ ]:
# ハイブリッド検索クエリ
hybrid_query_text = "年次報告書"

print(f"🔀 ハイブリッド検索: '{hybrid_query_text}'\n")

if openai_client:
    try:
        # クエリの埋め込みを生成
        query_vector = generate_embedding(hybrid_query_text)
        
        if query_vector:
            # ハイブリッド検索の実行
            vector_query = VectorizedQuery(
                vector=query_vector,
                k_nearest_neighbors=5,
                fields="contentVector"
            )
            
            results = search_client.search(
                search_text=hybrid_query_text,  # キーワード検索も同時実行
                vector_queries=[vector_query],
                top=5,
                select=["id", "title", "content", "url", "contentType", "lastModified"]
            )
            
            print_search_results(results, max_results=5, show_content=True)
        else:
            print("❌ 埋め込みベクトルの生成に失敗しました")
    
    except Exception as e:
        print(f"⚠️ ハイブリッド検索エラー: {e}")
else:
    print("ℹ️ OpenAI クライアントが利用できません。ハイブリッド検索をスキップします。")

## 9. ACL フィルタリング

ユーザーまたはグループの権限に基づいて検索結果をフィルタリングします。

**実装例**: 特定のユーザーがアクセス可能なドキュメントのみを検索

In [ ]:
# ACL フィルタリング
# 実際の環境では、ログインユーザーの Object ID を取得
user_oid = "example-user-oid-12345"  # サンプル値
user_groups = ["example-group-oid-67890"]  # サンプル値

search_query_acl = "予算"

print(f"🔒 ACL フィルタリング検索: '{search_query_acl}'")
print(f"   ユーザー OID: {user_oid}")
print(f"   グループ OID: {user_groups}\n")

try:
    # ACL フィルタ式の構築
    # ユーザーまたはグループのいずれかに一致するドキュメントを検索
    filter_parts = []
    
    # ユーザーフィルタ
    if user_oid:
        filter_parts.append(f"search.in('{user_oid}', aclUsers, ',')")
    
    # グループフィルタ
    for group_oid in user_groups:
        filter_parts.append(f"search.in('{group_oid}', aclGroups, ',')")
    
    # OR条件で結合
    filter_expression = " or ".join(filter_parts) if filter_parts else None
    
    if filter_expression:
        print(f"フィルタ式: {filter_expression}\n")
        
        results = search_client.search(
            search_text=search_query_acl,
            filter=filter_expression,
            top=5,
            select=["id", "title", "content", "url", "contentType", "lastModified", "aclUsers", "aclGroups"]
        )
        
        print_search_results(results, max_results=5, show_content=True)
    else:
        print("⚠️ ACL フィルタが設定されていません")

except Exception as e:
    print(f"⚠️ ACL フィルタ検索エラー: {e}")
    print("ℹ️ ACL フィルタを使用するには、インデックスに aclUsers と aclGroups フィールドが必要です。")

## 10. ACL フィルタの効果確認

ACL フィルタありとなしで結果を比較します。

In [ ]:
comparison_query = "報告書"

print(f"🔍 比較検索: '{comparison_query}'\n")

try:
    # フィルタなし
    print("="*80)
    print("🌐 ACL フィルタなし (すべてのドキュメント)")
    print("="*80)
    
    results_no_filter = search_client.search(
        search_text=comparison_query,
        top=3,
        include_total_count=True,
        select=["id", "title", "url"]
    )
    
    total_no_filter = results_no_filter.get_count()
    print(f"\n総件数: {total_no_filter}")
    print_search_results(results_no_filter, max_results=3, show_content=False)
    
    # フィルタあり
    print("\n" + "="*80)
    print("🔒 ACL フィルタあり (特定ユーザーのみ)")
    print("="*80)
    
    filter_expression = f"search.in('{user_oid}', aclUsers, ',')"
    
    results_with_filter = search_client.search(
        search_text=comparison_query,
        filter=filter_expression,
        top=3,
        include_total_count=True,
        select=["id", "title", "url"]
    )
    
    total_with_filter = results_with_filter.get_count()
    print(f"\n総件数: {total_with_filter}")
    print_search_results(results_with_filter, max_results=3, show_content=False)
    
    print("\n" + "="*80)
    print(f"📊 フィルタによる絞り込み: {total_no_filter} → {total_with_filter} 件")
    print("="*80)

except Exception as e:
    print(f"⚠️ 比較検索エラー: {e}")

## まとめ

このノートブックでは、以下の検索パターンを試しました:

1. ✅ **キーワード検索**: 基本的なテキストマッチング
2. ✅ **セマンティック検索**: 意味理解に基づく検索
3. ✅ **ベクター検索**: 埋め込みベクトルによる類似検索
4. ✅ **ハイブリッド検索**: キーワード + ベクターの組み合わせ
5. ✅ **ACL フィルタ**: ユーザー権限による検索結果の制御

### 次のステップ

- 実際のユーザー認証と組み合わせて ACL フィルタを実装
- RAG (Retrieval-Augmented Generation) パターンでの活用
- 検索品質の評価と改善

### 参考リソース

- [Azure AI Search クエリ構文](https://learn.microsoft.com/azure/search/query-simple-syntax)
- [セマンティック検索](https://learn.microsoft.com/azure/search/semantic-search-overview)
- [ベクター検索](https://learn.microsoft.com/azure/search/vector-search-overview)